# ETL version 2

In [1]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime
from dateutil.relativedelta import relativedelta
import boto3
import json

In [2]:
# 📌 Cargar variables de entorno
load_dotenv()

# 📌 Obtener los tokens desde .env
BANXICO_TOKEN = os.getenv("BANXICO_TOKEN")
AWS_ACCESS_KEY = os.getenv("AWS_ACCESS_KEY")
AWS_SECRET_KEY = os.getenv("AWS_SECRET_KEY")
S3_BUCKET_NAME = os.getenv("S3_BUCKET_NAME")

if not BANXICO_TOKEN:
    raise ValueError("❌ Error: BANXICO_TOKEN no encontrado en .env")

print("✅ Token de Banxico cargado correctamente.")

# 📌 Configurar fechas desde 2020 hasta hoy
initial_date = "2015-01-01"
final_date = datetime.today().strftime("%Y-%m-%d")

# 📌 Configurar cliente de Amazon S3
s3_client = boto3.client(
    "s3",
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
)

✅ Token de Banxico cargado correctamente.


## Extract 

In [3]:
def extract_banxico_data(serie_id, initial_date, final_date):
    """
    Extrae datos de la API de Banxico y los devuelve en un DataFrame.
    """
    url = f"https://www.banxico.org.mx/SieAPIRest/service/v1/series/{serie_id}/datos/{initial_date}/{final_date}?token={BANXICO_TOKEN}"
    response = requests.get(url)

    if response.status_code != 200:
        print(f"❌ Error {response.status_code}: {response.text}")
        return None

    data = response.json()
    
    if "bmx" not in data or "series" not in data["bmx"] or not data["bmx"]["series"]:
        print("⚠️ No se encontraron datos en la respuesta de Banxico.")
        return None

    series_data = data["bmx"]["series"][0]["datos"]

    if not series_data:
        print("⚠️ No hay datos disponibles en este rango de fechas.")
        return None

    # 📌 Convertir datos a DataFrame
    df = pd.DataFrame(series_data)
    df.columns = ["timestamp", "value"]

    # 📌 Convertir fechas correctamente
    df["timestamp"] = pd.to_datetime(df["timestamp"], format="%d/%m/%Y", dayfirst=True)

    # 📌 Convertir valores a numérico
    df["value"] = pd.to_numeric(df["value"], errors="coerce")

    return df


In [4]:
# 📌 Extraer Tipo de Cambio
df_tipo_cambio = extract_banxico_data("SF43718", initial_date, final_date)
df_tipo_cambio.to_csv("tipo_de_cambio.csv", index=False)

# 📌 Extraer Tasa de Interés
df_tasa_interes = extract_banxico_data("SF282", initial_date, final_date)
df_tasa_interes.to_csv("tasa_de_interes.csv", index=False)

# 📌 Extraer INPC desde Banxico
df_inpc = extract_banxico_data("SP1", initial_date, final_date)
df_inpc.to_csv("inpc.csv", index=False)

print("✅ Datos guardados localmente en CSV.")


✅ Datos guardados localmente en CSV.


---

## Transform

In [5]:
print(df_inpc.head())
print(df_tasa_interes.head())
print(df_tipo_cambio.head())

   timestamp      value
0 2015-01-01  87.110103
1 2015-02-01  87.275377
2 2015-03-01  87.630717
3 2015-04-01  87.403840
4 2015-05-01  86.967366
   timestamp  value
0 2015-01-01   2.67
1 2015-02-01   2.81
2 2015-03-01   3.04
3 2015-04-01   2.97
4 2015-05-01   2.98
   timestamp    value
0 2015-01-02  14.8290
1 2015-01-05  14.9469
2 2015-01-06  14.8479
3 2015-01-07  14.7936
4 2015-01-08  14.6274


In [6]:
print(df_inpc.dtypes)
print(df_tasa_interes.dtypes)
print(df_tipo_cambio.dtypes)


timestamp    datetime64[ns]
value               float64
dtype: object
timestamp    datetime64[ns]
value               float64
dtype: object
timestamp    datetime64[ns]
value               float64
dtype: object


In [7]:
# 📌 Renombrar columnas para coherencia
df_tipo_cambio.rename(columns={"timestamp": "date", "value": "tipo_de_cambio"}, inplace=True)
df_tasa_interes.rename(columns={"timestamp": "date", "value": "tasa_de_interes"}, inplace=True)
df_inpc.rename(columns={"timestamp": "date", "value": "inpc"}, inplace=True)

In [8]:
# 📌 Crear la columna con el INPC rezagado 12 meses
df_inpc["inpc_lag_12"] = df_inpc["inpc"].shift(12)

# 📌 Calcular la inflación anualizada en porcentaje
df_inpc["inflacion"] = 100 * (df_inpc["inpc"] / df_inpc["inpc_lag_12"] - 1)

# 📌 Eliminar filas con valores NaN (primeros 12 meses no tienen inpc_lag_12)
df_inpc = df_inpc.dropna()

print("✅ Inflación anualizada calculada correctamente")

✅ Inflación anualizada calculada correctamente


In [9]:
# 📌 Convertir las fechas a formato datetime estándar
df_tipo_cambio["date"] = pd.to_datetime(df_tipo_cambio["date"], format="%Y-%m-%d")
df_tasa_interes["date"] = pd.to_datetime(df_tasa_interes["date"], format="%Y-%m-%d")
df_inpc["date"] = pd.to_datetime(df_inpc["date"], format="%Y-%m-%d")

print("✅ Fechas convertidas correctamente")

# 📌 Convertir las fechas a solo "AAAA-MM" (a nivel mensual)
df_tipo_cambio["date"] = df_tipo_cambio["date"].dt.to_period("M")
df_tasa_interes["date"] = df_tasa_interes["date"].dt.to_period("M")
df_inpc["date"] = df_inpc["date"].dt.to_period("M")

print("✅ Fechas convertidas a nivel mensual")


✅ Fechas convertidas correctamente
✅ Fechas convertidas a nivel mensual


In [10]:
# 📌 Para el tipo de cambio, tomamos el promedio mensual
df_tipo_cambio = df_tipo_cambio.groupby("date").agg({"tipo_de_cambio": "mean"}).reset_index()

print("✅ Tipo de cambio agregado a nivel mensual")

✅ Tipo de cambio agregado a nivel mensual


In [12]:
df_inpc

,date,inpc,inpc_lag_12,inflacion
12,2016-01,89.386381,87.110103,2.613104
13,2016-02,89.777781,87.275377,2.867251
14,2016-03,89.910001,87.630717,2.601010
15,2016-04,89.625278,87.403840,2.541579
16,2016-05,89.225615,86.967366,2.596663
...,...,...,...,...
117,2024-10,136.828000,130.609000,4.761540
118,2024-11,137.424000,131.445000,4.548671
119,2024-12,137.949000,132.373000,4.212339
120,2025-01,138.343000,133.555000,3.585040


In [13]:
df_tasa_interes

,date,tasa_de_interes
0,2015-01,2.67
1,2015-02,2.81
2,2015-03,3.04
3,2015-04,2.97
4,2015-05,2.98
...,...,...
117,2024-10,10.24
118,2024-11,10.05
119,2024-12,9.85
120,2025-01,9.86


In [14]:
df_tipo_cambio

,date,tipo_de_cambio
0,2015-01,14.692586
1,2015-02,14.921342
2,2015-03,15.228338
3,2015-04,15.226180
4,2015-05,15.264470
...,...,...
118,2024-11,20.345490
119,2024-12,20.266055
120,2025-01,20.549018
121,2025-02,20.457137


In [22]:
# Convertir 'date' de period[M] a datetime64 en cada DataFrame
df_inpc["date"] = df_inpc["date"].dt.to_timestamp()
df_tasa_interes["date"] = df_tasa_interes["date"].dt.to_timestamp()
df_tipo_cambio["date"] = df_tipo_cambio["date"].dt.to_timestamp()

# Asegurar que el formato sea YYYY-MM-DD
df_inpc["date"] = df_inpc["date"].dt.strftime('%Y-%m-%d')
df_tasa_interes["date"] = df_tasa_interes["date"].dt.strftime('%Y-%m-%d')
df_tipo_cambio["date"] = df_tipo_cambio["date"].dt.strftime('%Y-%m-%d')

# Verificar cambios
print(df_inpc.dtypes)
print(df_tasa_interes.dtypes)
print(df_tipo_cambio.dtypes)


date            object
inpc           float64
inpc_lag_12    float64
inflacion      float64
dtype: object
date                object
tasa_de_interes    float64
dtype: object
date               object
tipo_de_cambio    float64
dtype: object


In [23]:
# Convertir a datetime64 antes de exportar
df_inpc["date"] = pd.to_datetime(df_inpc["date"])
df_tasa_interes["date"] = pd.to_datetime(df_tasa_interes["date"])
df_tipo_cambio["date"] = pd.to_datetime(df_tipo_cambio["date"])

# Verificar cambios
print(df_inpc.dtypes)
print(df_tasa_interes.dtypes)
print(df_tipo_cambio.dtypes)


date           datetime64[ns]
inpc                  float64
inpc_lag_12           float64
inflacion             float64
dtype: object
date               datetime64[ns]
tasa_de_interes           float64
dtype: object
date              datetime64[ns]
tipo_de_cambio           float64
dtype: object


---

## Load 

In [16]:
# Guardar en local
df_inpc.to_csv("inpc_mensual.csv", index=False)
df_tasa_interes.to_csv("tasa_interes_mensual.csv", index=False)
df_tipo_cambio.to_csv("tipo_cambio_mensual.csv", index=False)

print("✅ Datos guardados localmente en CSV.")


✅ Datos guardados localmente en CSV.


In [17]:
# 📌 Guardar en S3
s3_client = boto3.client("s3")

# Probar conexión
response = s3_client.list_buckets()
for bucket in response["Buckets"]:
    print(f"✅ Bucket encontrado: {bucket['Name']}")
# 📌 Probar conexión
try:
    response = s3_client.list_objects_v2(Bucket="banxico-hw", Prefix="raw/")
    for obj in response.get("Contents", []):
        print(f"📂 Archivo en S3: {obj['Key']} - Tamaño: {obj['Size']} bytes")
except Exception as e:
    print(f"❌ Error al conectar con S3: {e}")

✅ Bucket encontrado: arquitectura-athena-queries-sofia
✅ Bucket encontrado: banxico-hw
✅ Bucket encontrado: itam-analytics-sofia
✅ Bucket encontrado: sofia-temp-flights


In [20]:
def upload_to_s3(file_name, bucket, folder):
    """
    Sube un archivo local a Amazon S3 en la carpeta especificada.
    """
    object_name = f"{folder}/{file_name}"
    try:
        s3_client.upload_file(file_name, bucket, object_name)
        print(f"✅ Archivo {file_name} subido a S3 en {object_name}")
    except Exception as e:
        print(f"❌ Error al subir {file_name}: {e}")

# 📌 Subir los archivos a S3
upload_to_s3("tipo_cambio_mensual.csv", "banxico-hw", "tipo_de_cambio")
upload_to_s3("tasa_interes_mensual.csv", "banxico-hw", "tasa_de_interes")
upload_to_s3("inpc_mensual.csv", "banxico-hw", "inflacion")

print("✅ Todos los archivos han sido subidos a S3 en la carpeta 'raw/'.")


✅ Archivo tipo_cambio_mensual.csv subido a S3 en tipo_de_cambio/tipo_cambio_mensual.csv
✅ Archivo tasa_interes_mensual.csv subido a S3 en tasa_de_interes/tasa_interes_mensual.csv
✅ Archivo inpc_mensual.csv subido a S3 en inflacion/inpc_mensual.csv
✅ Todos los archivos han sido subidos a S3 en la carpeta 'raw/'.
